In [1]:
import numpy as np
import ray
import scipy
import math
import datetime
#import mkl
import numpy.linalg as la
import os
from sklearn.linear_model import RidgeCV
import pickle

In [2]:
ray.init(num_cpus=48, redis_password="123456")

2020-01-19 18:23:05,451	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2020-01-19_18-23-05_24234/logs.
2020-01-19 18:23:05,561	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:31484 to respond...
2020-01-19 18:23:05,678	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:34493 to respond...
2020-01-19 18:23:05,683	INFO services.py:804 -- Starting Redis shard with 10.0 GB max memory.
2020-01-19 18:23:05,704	INFO node.py:483 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2020-01-19_18-23-05_24234/logs.
2020-01-19 18:23:05,707	WARNING services.py:1279 -- Warning: Capping object memory store to 20.0GB. To increase this further, specify `object_store_memory` when calling ray.init() or ray start.
2020-01-19 18:23:05,709	INFO services.py:1427 -- Starting the Plasma object store with 20.0 GB memory using /dev/shm.


{'node_ip_address': '169.229.48.80',
 'redis_address': '169.229.48.80:31484',
 'object_store_address': '/tmp/ray/session_2020-01-19_18-23-05_24234/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-01-19_18-23-05_24234/sockets/raylet',
 'webui_url': None}

In [3]:
def eigs(M):
    
    eigenValues, eigenVectors = la.eig(M)

    idx = eigenValues.argsort()[::-1]   
    eigenValues = eigenValues[idx]
    eigenVectors = eigenVectors[:,idx]
    
    return eigenValues, eigenVectors

In [4]:
def gen_train_model(d, r, T, train_n):
    
    u, s, v = la.svd(np.random.normal(size=(d, r)))
    B = u[:, :r]
    
    train_alphas = [np.random.normal(size=r, scale=1/math.sqrt(r)) for i in range(T)]
    train_data=[]
    for i in range(T):
        X=np.random.normal(size=(train_n, d))
        y = X @ B @ train_alphas[i] + np.random.normal(size=train_n)
        train_data.append((X, y))
        
    return train_data, B, train_alphas

In [5]:
def gen_test_model(d, r, B, test_n):
    
    alpha = np.random.normal(size=r, scale=1/math.sqrt(r))

    X=np.random.normal(size=(test_n, d))
    y = X @ B @ alpha + np.random.normal(size=test_n)
        
    return (X, y), alpha

In [6]:
def MoM(train_data):
    
    T = len(train_data)
    d = train_data[0][0].shape[1]
    
    total_n=0
    M = np.zeros(shape=(d, d))
    for i in range(T):
        data = train_data[i]
        X, y = data
        num = y.shape[0]
        total_n += num
        scaled_X = (X.T * y).T
        M += (scaled_X).T @ scaled_X
    M = 1/float(total_n) * M
    
    return M

In [7]:
def rPCA(M, r):
    
    eigVals, eigVecs = eigs(M)
    
    return eigVecs[:, :r], eigVecs[:, r:]

In [8]:
def MetaLR(train_data, r, test_data):
    
    T = len(train_data)
    d = train_data[0][0].shape[1]
    
    M_est = MoM(train_data)
    B1, B2 = rPCA(M_est, r)
    
    X,y = test_data
    X_low = X @ B1
    alpha_LR = LR((X_low, y))
    beta_LR = B1 @ alpha_LR
    
    alpha_RR = ridge_regression((X_low, y))
    beta_RR = B1 @ alpha_RR
    
    return B1, beta_LR, beta_RR

In [9]:
def LR(test_data):
    
    X, y = test_data
    beta_LR = la.pinv((X.T @ X)) @ X.T @ y
    
    return beta_LR

In [10]:
def ridge_regression(test_data):
    
    X, y = test_data
    reg = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1, 10], fit_intercept=False)
    reg.fit(test_data[0], test_data[1])

    return reg.coef_

In [11]:
@ray.remote
def run_expt(d, r, T, train_n, test_n):
    
    #mkl.set_num_threads(1)
    train_data, B, train_alphas = gen_train_model(d=d, r=r, T=T, train_n=train_n)
    test_data, alpha_test = gen_test_model(d, r, B, test_n)
    B_meta, beta_meta_LR, beta_meta_RR = MetaLR(train_data, r, test_data)
    beta_LR = LR(test_data)
    beta_RR = ridge_regression(test_data)

    beta_true = B @ alpha_test

    return np.linalg.norm(beta_meta_LR-beta_true), np.linalg.norm(beta_meta_RR-beta_true), np.linalg.norm(beta_LR-beta_true), np.linalg.norm(beta_RR-beta_true)

In [12]:
def run_parallel_expt(d, r, T, train_n, test_n, reps):
    
    meta_LR_errs=[]
    meta_RR_errs=[]
    LR_errs=[]
    ridge_errs=[]

    data = ray.get([run_expt.remote(d, r, T, train_n, test_n) for num in range(reps)])
    meta_LR_errs, meta_RR_errs, LR_errs, ridge_errs = zip(*data)
    
    return meta_LR_errs, meta_RR_errs, LR_errs, ridge_errs

In [13]:
d=250
r=5
train_n=50
test_n=500
reps=50

In [14]:
T_list = [200, 400, 800, 1600, 3200, 6400, 12800, 12800*2]

In [15]:
def collect_data(d, r, T_list, train_n, test_n, reps):
    
    metaLRmus=[]
    metaLRstd=[]
    
    metaRRmus=[]
    metaRRstd=[]
    
    LRmus=[]
    LRstd=[]
    
    ridgemus=[]
    ridgestd=[]
    
    for t in T_list:
        print(t)
        meta_LR_errs, meta_RR_errs, LR_errs, ridge_errs = run_parallel_expt(d, r, t, train_n, test_n, reps)

        metaLRmus.append(np.mean(meta_LR_errs))
        metaLRstd.append(np.std(meta_LR_errs)) 

        metaRRmus.append(np.mean(meta_RR_errs))
        metaRRstd.append(np.std(meta_RR_errs)) 


        LRmus.append(np.mean(LR_errs))
        LRstd.append(np.std(LR_errs)) 

        ridgemus.append(np.mean(ridge_errs))
        ridgestd.append(np.std(ridge_errs)) 
        
    return (metaLRmus, metaLRstd), (metaRRmus, metaRRstd), (LRmus, LRstd), (ridgemus, ridgestd)

In [16]:
meta_LR_errs, meta_RR_errs, LR_errs, ridge_errs = collect_data(d, r, T_list, train_n, test_n, reps)

200
400
800
1600
3200
6400
12800
25600


In [17]:
save_data = {"meta_LR_errs" : meta_LR_errs, "meta_RR_errs" :  meta_RR_errs, "LR_errs" : LR_errs, "ridge_errs" : ridge_errs}

In [18]:
save_data["T_list"] =  T_list
save_data["d"] = d
save_data["r"] = r
save_data["train_n"] = train_n
save_data["test_n"] = test_n
save_data["reps"] = reps

In [19]:
meta_LR_errs

([0.9264854523960608,
  0.8922002608193322,
  0.765839597383465,
  0.5679277123271294,
  0.4220414105474616,
  0.304358314761279,
  0.21582544763254174,
  0.17455443231034262],
 [0.3014498912022357,
  0.29950957049521343,
  0.2607231340474149,
  0.1748487069565234,
  0.13963699046275063,
  0.09523098115684553,
  0.05429350945721333,
  0.039560454963484766])

In [20]:
LR_errs

([1.013992320106808,
  0.9921867798552348,
  0.9977008597166089,
  1.0046957719814231,
  0.9935672267783785,
  1.0068740897589308,
  0.9973984088280997,
  0.9819730986244446],
 [0.06957946777209406,
  0.06283619283589094,
  0.060839528814186576,
  0.06266060419149605,
  0.06787414676619481,
  0.06611949642184788,
  0.0638938508903859,
  0.065508098814329])

In [21]:
params = "d="+str(d)+",r="+str(r)+",train_n="+str(train_n)+",test_n="+str(test_n)

In [22]:
file_name = "Meta,"+str(params)+".pickle"
folder_name = "Data"
file_path = os.path.join(folder_name, file_name)
pickle.dump(save_data, open(file_path, "wb"))